In [1]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as func
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as trans

In [2]:
 
download_url("https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz", '.')

with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
    
print(os.listdir('./data/cifar10'))
classes = os.listdir('./data/cifar10' + "/train")
print(classes)

  0%|          | 0/135107811 [00:00<?, ?it/s]

['train', 'test']
['frog', 'dog', 'bird', 'airplane', 'ship', 'truck', 'deer', 'horse', 'cat', 'automobile']


In [25]:
trainIF = ImageFolder('./data/cifar10/train', trans.Compose([trans.RandomCrop(32, padding=4, padding_mode='reflect'),
                         trans.AutoAugment(policy=trans.AutoAugmentPolicy.CIFAR10),
                         trans.RandomHorizontalFlip(),
                         trans.ToTensor(), 
                         trans.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010),inplace=True)]))
validIF = ImageFolder('./data/cifar10/test', trans.Compose([trans.ToTensor(), trans.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]))

In [26]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)
    
device = get_default_device()
device

device(type='cuda')

In [27]:
batchSize = 400
trainDL = DataLoader(trainIF, batchSize, shuffle=True, num_workers = 2, pin_memory=True)
validDL = DataLoader(validIF, batchSize*2, num_workers = 2, pin_memory=True)

In [29]:
trainDL = DeviceDataLoader(trainDL, device)
validDL = DeviceDataLoader(validDL, device)

In [30]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        channelsIn = 3
        channelsOut = 3
        kernel_size = 3
        stride = 1
        padding = 1
        self.conv1 = nn.Conv2d(channelsIn, channelsOut, kernel_size, stride, padding)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(channelsIn, channelsOut, kernel_size, stride, padding)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        return self.relu2(out) + x 
    


simpleResnet = to_device(SimpleResidualBlock(), device)

for images, labels in trainDL:
    out = simpleResnet(images)
    print(out.shape)
    break
    
del simpleResnet, images, labels
torch.cuda.empty_cache()



torch.Size([400, 3, 32, 32])


In [31]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = func.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = func.cross_entropy(out, labels)
        acc = accuracy(out, labels) 
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))
     
    
    
def convBlock(ChannelsIn, channelsOut, pool=False):
    layers = [nn.Conv2d(ChannelsIn, channelsOut, kernel_size=3, padding=1), 
            nn.BatchNorm2d(channelsOut), 
            nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, channelsIn, numClasses, channelsOut):
        super().__init__()
        
        self.conv1 = convBlock(channelsIn, channelsOut)
        channelsIn = channelsOut
        self.conv2 = convBlock(channelsIn, channelsOut*2, pool=True)
        self.res1 = nn.Sequential(convBlock(channelsIn*2, channelsOut*2), convBlock(channelsIn*2, channelsOut*2))
        
        self.conv3 = convBlock(channelsIn*2, channelsOut*4, pool=True)
        self.conv4 = convBlock(channelsIn*4, channelsOut*8, pool=True)
        self.res2 = nn.Sequential(convBlock(channelsIn*8, channelsOut*8), convBlock(channelsIn*8, channelsOut*8))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Linear(512, numClasses))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [32]:
model = to_device(ResNet9(3, 10, 64), device)
model

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

In [33]:
@torch.no_grad()
def evaluate(model, validDL):
    model.eval()
    outputs = [model.validation_step(batch) for batch in validDL]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def customOptimizer(model, max_lr, weightDecay, opt_func):
    return opt_func(model.parameters(), max_lr, weight_decay=weightDecay)

def fit_one_cycle(train_loader, val_loader, epochs, max_lr, model, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            lrs.append(get_lr(optimizer))
            sched.step()
        
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch+1, result)
        history.append(result)
    return history


In [34]:
import gc
import os
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'

gc.collect()

torch.cuda.empty_cache()
history = [evaluate(model, validDL)]
history

[{'val_loss': 2.3034961223602295, 'val_acc': 0.10096153616905212}]

In [35]:
%%time
history += fit_one_cycle(trainDL, validDL, 25, 0.009, model,
                             grad_clip=0.1, 
                             weight_decay=1e-4, 
                             opt_func=torch.optim.Adam)

Epoch [1], last_lr: 0.00073, train_loss: 1.6815, val_loss: 1.2268, val_acc: 0.5649
Epoch [2], last_lr: 0.00178, train_loss: 1.1932, val_loss: 0.8052, val_acc: 0.7231
Epoch [3], last_lr: 0.00334, train_loss: 1.0168, val_loss: 0.8428, val_acc: 0.7247
Epoch [4], last_lr: 0.00512, train_loss: 0.9504, val_loss: 1.0585, val_acc: 0.6763
Epoch [5], last_lr: 0.00684, train_loss: 0.9432, val_loss: 0.7586, val_acc: 0.7430
Epoch [6], last_lr: 0.00817, train_loss: 0.8517, val_loss: 0.6521, val_acc: 0.7832
Epoch [7], last_lr: 0.00891, train_loss: 0.8139, val_loss: 0.6965, val_acc: 0.7710
Epoch [8], last_lr: 0.00898, train_loss: 0.7583, val_loss: 0.7722, val_acc: 0.7587
Epoch [9], last_lr: 0.00884, train_loss: 0.6833, val_loss: 0.5664, val_acc: 0.8083
Epoch [10], last_lr: 0.00855, train_loss: 0.6514, val_loss: 0.5555, val_acc: 0.8149
Epoch [11], last_lr: 0.00814, train_loss: 0.6311, val_loss: 0.5023, val_acc: 0.8389
Epoch [12], last_lr: 0.00761, train_loss: 0.5995, val_loss: 0.4878, val_acc: 0.8363
E

In [36]:
torch.save(model.state_dict(), 'zavrsni.pth')

In [ ]:
model = torch.load('zavrsni.pth')
model.eval()